In [ ]:
#importing required packages for proposed system
import numpy as np

In [ ]:
import cv2              #OpenCV package

In [ ]:
import img2pdf

In [ ]:
from collections import deque
from PIL import Image   #convert img to pdf

In [ ]:
# Defining boundaries for upper and lower color considered to be "Blue"
blueLower = np.array([100, 60, 60])
blueUpper = np.array([140, 255, 255])

In [ ]:
# Define a 5x5 kernel for erosion and dilation
kernel = np.ones((5, 5), np.uint8)

In [ ]:
# Setup deques to store separate colors in separate arrays
b_points = [deque (maxlen = 512)]
g_points = [deque (maxlen = 512)]
r_points = [deque (maxlen = 512)]
y_points = [deque (maxlen = 512)]

In [ ]:
b_index = 0
g_index = 0
r_index = 0
y_index = 0

In [ ]:
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255)]
colorIndex = 0

In [ ]:
# Setup the Paint interface
paintWindow = np.zeros((471,636,3)) + 255
paintWindow = cv2.rectangle(paintWindow, (40,1), (140,65), (0,0,0), 2)
paintWindow = cv2.rectangle(paintWindow, (160,1), (255,65), colors[0], -1)
paintWindow = cv2.rectangle(paintWindow, (275,1), (370,65), colors[1], -1)
paintWindow = cv2.rectangle(paintWindow, (390,1), (485,65), colors[2], -1)
paintWindow = cv2.rectangle(paintWindow, (505,1), (600,65), colors[3], -1)

In [ ]:
cv2.putText(paintWindow, "CLEAR ALL", (49, 33), cv2.FONT_HERSHEY_SIMPLEX,0.5, (0, 0, 0), 2, cv2.LINE_AA)
cv2.putText(paintWindow, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5,       (255, 255, 255), 2, cv2.LINE_AA)       #text on paint window
cv2.putText(paintWindow, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5,    (255, 255, 255), 2, cv2.LINE_AA)         #text on paint window
cv2.putText(paintWindow, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5,         (255, 255, 255), 2, cv2.LINE_AA)      #text on paint window
cv2.putText(paintWindow, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,150), 2, cv2.LINE_AA)             #text on paint window
cv2.namedWindow('Paint', cv2.WINDOW_AUTOSIZE)

In [ ]:
# Load the video
webcamera = cv2.VideoCapture(0)

In [ ]:
# For Keep - looping
while True:
    # To grab current paintWindow/frame
    (grab, frame) = webcamera.read()            
    frame = cv2.flip(frame, 1)                  #frame setting
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

     # Add the coloring options to the frame
    frame = cv2.rectangle(frame, (40,1), (140,65), (122,122,122), -1)
    frame = cv2.rectangle(frame, (160,1), (255,65), colors[0], -1)
    frame = cv2.rectangle(frame, (275,1), (370,65), colors[1], -1)
    frame = cv2.rectangle(frame, (390,1), (485,65), colors[2], -1)
    frame = cv2.rectangle(frame, (505,1), (600,65), colors[3], -1)
   

    cv2.putText(frame, "CLEAR ALL", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)       #text on track screen
    cv2.putText(frame, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)           #text on track screen
    cv2.putText(frame, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)          #text on track screen
    cv2.putText(frame, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)            #text on track screen
    cv2.putText(frame, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (150,150,150), 2, cv2.LINE_AA)           #text on track screen

    # Test to check whether we have reached the end of the video
    if not grab:
        break

    # To determine that what pixels fall within blue boundaries and then blur the binary image
    blueMask = cv2.inRange(hsv, blueLower, blueUpper)
    blueMask = cv2.erode(blueMask, kernel, iterations=2)
    blueMask = cv2.morphologyEx(blueMask, cv2.MORPH_OPEN, kernel)
    blueMask = cv2.dilate(blueMask, kernel, iterations=1)

     # To Find contours in the image
    (cnts, _) = cv2.findContours(blueMask.copy(), cv2.RETR_EXTERNAL,
        cv2.CHAIN_APPROX_SIMPLE)
    center = None

    # To check whether any contours were found
    if len(cnts) > 0:                   #check for contour lengths
        # To sort the contours, then find the largest one among.
        # can assume this contour proportional to the area of the bottle cap
        cnt = sorted(cnts,  reverse = True, key = cv2.contourArea)[0]
        
        # To get the radius of circle enclosed around the contour found
        ((x, y), radius) = cv2.minEnclosingCircle(cnt)
        
        # To Draw circle around the found contour
        cv2.circle(frame, ( int(x), int(y) ), int(radius), (0, 255, 255), 2)
        
        # To Get moments to calculate the center of contour (in this case Circle)
        M = cv2.moments(cnt)
        center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))

#To match with the colors used for background distortion
        if center[1] <= 65:
            if 40 <= center[0] <= 140:                  # Clear All
                b_points = [deque (maxlen = 512)]
                g_points = [deque (maxlen = 512)]
                r_points = [deque (maxlen = 512)]
                y_points = [deque (maxlen = 512)]
               
                b_index = 0
                g_index = 0
                r_index = 0
                y_index = 0

                paintWindow[67:,:,:] = 255
            elif 160 <= center[0] <= 255:
                colorIndex = 0                  # Blue
            elif 390 <= center[0] <= 485:
                colorIndex = 2                  # Red
            elif 275 <= center[0] <= 370:
                colorIndex = 1  
            elif 505 <= center[0] <= 600:
                colorIndex = 3              # Green
            # elif 505 <= center[0] <= 600:
            #     colorIndex = 3                    # Yellow
    
        else:
            if colorIndex == 0:
                b_points[b_index].appendleft(center)        # Blue
            elif colorIndex == 1:
                g_points[g_index].appendleft(center)        # Green
            elif colorIndex == 2:                   
                r_points[r_index].appendleft(center)        #Red
            elif colorIndex == 3:               
                y_points[y_index].appendleft(center)        #Yellow
    
     #To Append next deque if no contours are detected (i.e., bottle cap reversed)
    else:
        b_points.append(deque(maxlen = 512))
        b_index += 1
        g_points.append(deque(maxlen = 512))
        g_index += 1
        r_points.append(deque(maxlen = 512))
        r_index += 1
        y_points.append(deque(maxlen = 512))
        y_index += 1

    
# To Draw the lines of all included colors (Blue, Green, Red and Yellow)
    points = [b_points, g_points, r_points, y_points]
    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1, len(points[i][j])):
                if points[i][j][k - 1] is None or points[i][j][k] is None:
                    continue
                cv2.line(frame, points[i][j][k - 1], points[i][j][k], colors[i], 2)
                cv2.line(paintWindow, points[i][j][k - 1], points[i][j][k], colors[i], 2)

    #To  Show the Tracking frame and the paint Window screen
    cv2.imshow("Tracking", frame)
    cv2.imshow("Paint", paintWindow)

    #To save image as image
    k= cv2.waitKey(1)
    if k == ord("s"):
        cv2.imwrite('air-writing image.png',paintWindow)
        pdf_path=r"C:\1review"
        pdf = img2pdf.convert('air-writing image.png')
        file = open(pdf_path, "wb")
        file.write(pdf)
        # file.close()
   
    #To convert image as PDF
    elif k == ord("p"):
        cv2.imwrite('air-writing image.png',paintWindow)
       
        #converting image to pdf format
        image1=Image.open(r'C:\1review\air-writing image.png')
        im1=image1.convert('RGB')
        im1.save(r'C:\1review\air-writing pdf.pdf')
        file.close()
   
    # If key 'q' is pressed, it will stop the loop
    k= cv2.waitKey(1)
    if k & 0xFF == ord("q"):
        break

# To Clean up the web camera and close if any windows open
webcamera.release()
cv2.destroyAllWindows()

